In [1]:
import numpy as np
import scipy.linalg
import cudaq

import poly
import sym_qsp_opt
 
import matplotlib.pyplot as plt

In [2]:
# Classical Linear System Solver
class ClassicalSolver:
    def __init__(self, A, b):
        self.A = A
        self.b = b

    def solve(self):
        """
        Solves the linear system Ax = b using a classical solver.
        
        Returns:
            x: Solution vector.
        """
        x = scipy.linalg.solve(self.A, self.b)  # Using SciPy's solver for Ax = b
        return x

In [249]:
class QuantumSolverQSVT:
    def __init__(self, A, b, epsilon=1e-3, phases=None):
        """
        Initializes the QuantumSolverQSVT with matrix A and vector b.
        
        Parameters:
            A (numpy.ndarray): The coefficient matrix for the linear system.
            b (numpy.ndarray): The right-hand side vector of the linear system.
        """
        self.A = A
        self.b = b
        # TODO: alpha and kappa may need to be replaced by their respective upper bounds
        self.alpha = np.linalg.norm(A, ord=2)  # Spectral norm
        self.A_rescaled = self.A / self.alpha
        self.kappa = np.linalg.cond(A) # Condition number
        self.b_norm = np.linalg.norm(b) # Normalization 
        self.epsilon = epsilon # Desired presision
        self.N = len(b)
        self.n = int(np.log2(self.N))
        self.m = 1 # Number of ancillas for the blocj-encoding of A

        if phases is None:
        # if False:
            pg = poly.PolyOneOverX()
            pcoefs, scale = pg.generate(kappa=self.kappa, 
                                        epsilon=self.epsilon, 
                                        chebyshev_basis=True, 
                                        return_scale=True)
            # Using odd parity and instantiating desired coefficeints.
            parity = 1
            coef = pcoefs[parity::2]
            # Optimize to the desired function using Newton solver.
            crit = 1e-12
            (phases, err, total_iter, qsp_seq_opt) = sym_qsp_opt.newton_solver(coef, parity, crit=crit)
        self.phases = phases    
        self.d = len(self.phases) - 1

    def _block_encode_A(self):
        """
        Creates a block-encoded unitary matrix U that embeds A / alpha in its top-left block.
        """
        I = np.eye(self.A_rescaled.shape[0])
        top_right_block = scipy.linalg.sqrtm(I - self.A_rescaled @ self.A_rescaled.T.conj())
        bottom_left_block = scipy.linalg.sqrtm(I - self.A_rescaled.T.conj() @ self.A_rescaled)

        U = np.block([[self.A_rescaled, top_right_block],
                      [bottom_left_block, -1*self.A_rescaled.T.conj()]])
        return U

    def _state_preparation_b(self):
        """
        Creates a state-preparation unitary matrix Q s.t. Q|0> = |b> / ||b>|
        """
        B = np.column_stack((self.b / self.b_norm, np.random.randn(self.N, self.N - 1)))
        # Apply QR decomposition to B to get an orthonormal basis
        # The Q matrix from the QR decomposition will be unitary, and the first column will be b
        Q, _ = scipy.linalg.qr(B, mode='economic')
        return Q
    
    def _P(self, phi):
        dim = int(2**self.m)
        P = np.diag([np.exp(-1j*phi)] + [np.exp(1j*phi)]*(dim-1))
        return P

    def _build_circuit(self):
        num_qubits = self.n + self.m
        circuit = np.kron(self._P(self.phases[0]), np.eye(2))
        U_A = self._block_encode_A()
        
        for phi in self.phases[1:]:
            circuit = circuit @ U_A 
            circuit = circuit @ np.kron(self._P(phi), np.eye(2))
        
        return circuit
    
    def solve(self):
        """
        Solves the system Ax = b using a QSVT-based inversion (placeholder).
        
        Returns:
            x (numpy.ndarray): Solution vector, as if from a QSVT inversion.
        """
        x = 0
        return x

In [256]:
# # Define a test matrix A and vector b
A = np.array([[1,0],[0,1]], dtype=float)
b = np.array([1,0], dtype=float)
solver = QuantumSolverQSVT(A,b)

b=6, j0=7
[PolyOneOverX] minimum [-1.65200326] is at [-0.46474431]: normalizing
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 1000.
iter: 001 --- err: 2.048e-01
iter: 002 --- err: 2.792e-02
iter: 003 --- err: 1.129e-03
iter: 004 --- err: 2.220e-06
iter: 005 --- err: 8.684e-12
iter: 006 --- err: 3.137e-16
[sym_qsp] Stop criteria satisfied.


In [257]:
solver._build_circuit()

array([[ 0.95879364-0.28410342j,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.95879364-0.28410342j,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
        -0.95879364-0.28410342j,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.95879364-0.28410342j]])

In [258]:
# # Define a test matrix A and vector b
# A = np.array([[4, 1], [1, 3]], dtype=float)
# b = np.array([1, 2], dtype=float)

In [259]:
# @cudaq.kernel
# def kernel(A: np.ndarray, b: np.ndarray):
#     # A = np.array([[4, 1], [1, 3]], dtype=float)
#     # b = np.array([1, 2], dtype=float)
#     # solver = QuantumSolverQSVT(A, b)
#     # U_b = solver._state_preparation_b()
#     qubit_count = 2
#     qubits = cudaq.qvector(qubit_count)
#     h(qubits[0])
#     # cudaq.register_operation("custom_h", 1. / np.sqrt(2.) * np.array([1, 1, 1, -1]))

In [260]:
# A = np.array([[4, 1], [1, 3]])
# b = np.array([1, 2])
# solver = QuantumSolverQSVT(A, b)
# U_b = solver._state_preparation_b()

In [261]:
# args = ([1,2],[1,2,4])
# args = (A.tolist(),b.tolist())
# result = cudaq.sample(kernel, *args)